In [ ]:
import os
import sqlite3

import numpy as np
import pandas as pd

db_path = os.path.join(os.path.dirname("__file__"), "..", "..", "data", "ufc.db")

In [108]:
query = """
WITH cte1 AS (
    SELECT
        *,
        ROW_NUMBER() OVER (ORDER BY t1.rowid) AS rn
    FROM
        misc.gym_elo_ratings AS t1
),
cte2 AS (
    SELECT
        red_gym_id AS gym_id,
        rn,
        bout_id,
        event_id,
        blue_gym_id AS opp_gym_id,
        CASE
            WHEN red_gym_id = blue_gym_id THEN NULL
            WHEN red_outcome = 'W' THEN 1
            ELSE 0
        END AS win_flag,
        red_gym_elo AS gym_elo,
        blue_gym_elo AS opp_gym_elo
    FROM
        cte1
    WHERE
        red_gym_id IS NOT NULL
    UNION
    SELECT
        blue_gym_id AS gym_id,
        rn,
        bout_id,
        event_id,
        red_gym_id AS opp_gym_id,
        CASE
            WHEN red_gym_id = blue_gym_id THEN NULL
            WHEN red_outcome = 'L' THEN 1
            ELSE 0
        END AS win_flag,
        blue_gym_elo AS gym_elo,
        red_gym_elo AS opp_gym_elo
    FROM
        cte1
    WHERE
        blue_gym_id IS NOT NULL
),
cte3 AS (
    SELECT
        gym_id,
        ROW_NUMBER() OVER (PARTITION BY gym_id ORDER BY rn) AS bout_order,
        bout_id,
        ROW_NUMBER() OVER (PARTITION BY gym_id, event_id ORDER BY rn) AS bout_order_in_event,
        event_id,
        opp_gym_id,
        win_flag,
        gym_elo,
        opp_gym_elo
    FROM
        cte2
),
cte4 AS (
    SELECT
        gym_id,
        ROW_NUMBER() OVER (PARTITION BY gym_id ORDER BY bout_order) AS event_order,
        event_id,
        gym_elo
    FROM
        cte3
    WHERE
        bout_order_in_event = 1
),
cte5 AS (
    SELECT
        gym_id,
        event_order,
        event_id,
        gym_elo,
        AVG(gym_elo) OVER (
            PARTITION BY gym_id
            ORDER BY event_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_elo,
        gym_elo - LAG(gym_elo) OVER (
            PARTITION BY gym_id
            ORDER BY event_order
        ) AS gym_elo_change
    FROM
        cte4
),
event_level_feats AS (
    SELECT
        *,
        AVG(gym_elo_change) OVER (
            PARTITION BY gym_id
            ORDER BY event_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_elo_change
    FROM
        cte5
),
cte6 AS (
    SELECT
        t1.gym_id,
        t1.bout_order,
        t1.bout_id,
        t1.bout_order_in_event,
        t2.event_order,
        t1.event_id,
        t1.opp_gym_id,
        t1.win_flag,
        t1.gym_elo,
        t2.gym_avg_elo,
        t2.gym_elo_change,
        t2.gym_avg_elo_change,
        t1.opp_gym_elo
    FROM
        cte3 AS t1
    LEFT JOIN event_level_feats AS t2 ON t1.gym_id = t2.gym_id AND t1.event_id = t2.event_id
),
cte7 AS (
    SELECT
        gym_id,
        bout_order,
        bout_id,
        bout_order_in_event,
        event_order,
        event_id,
        opp_gym_id,
        AVG(win_flag) OVER (
            PARTITION BY gym_id
            ORDER BY bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_win_rate,
        gym_elo,
        gym_avg_elo,
        gym_elo_change,
        gym_avg_elo_change,
        AVG(opp_gym_elo) OVER (
            PARTITION BY gym_id
            ORDER BY bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_opp_elo,
        AVG(gym_elo - opp_gym_elo) OVER (
            PARTITION BY gym_id
            ORDER BY bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_elo_diff
    FROM
        cte6
),
cte8 AS (
    SELECT
        t1.*,
        AVG(t2.gym_win_rate) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_opp_win_rate,
        AVG(t1.gym_win_rate - t2.gym_win_rate) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_win_rate_diff,
        AVG(t2.gym_avg_elo) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_opp_avg_elo,
        AVG(t1.gym_avg_elo - t2.gym_avg_elo) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_avg_elo_diff,
        AVG(t2.gym_elo_change) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_opp_elo_change,
        AVG(t1.gym_elo_change - t2.gym_elo_change) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_elo_change_diff,
        AVG(t2.gym_avg_elo_change) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_opp_avg_elo_change,
        AVG(t1.gym_avg_elo_change - t2.gym_avg_elo_change) OVER (
            PARTITION BY t1.gym_id
            ORDER BY t1.bout_order
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS gym_avg_avg_elo_change_diff
    FROM
        cte7 AS t1
    LEFT JOIN cte7 AS t2 ON t1.gym_id = t2.opp_gym_id AND t1.bout_id = t2.bout_id AND t1.opp_gym_id = t2.gym_id
),
gym_feats AS (
    SELECT
        gym_id,
        event_id,
        gym_win_rate,
        gym_elo,
        gym_avg_elo,
        gym_elo_change,
        gym_avg_elo_change,
        gym_avg_opp_win_rate,
        gym_avg_win_rate_diff,
        gym_avg_opp_elo,
        gym_avg_elo_diff,
        gym_avg_opp_avg_elo,
        gym_avg_avg_elo_diff,
        gym_avg_opp_elo_change,
        gym_avg_elo_change_diff,
        gym_avg_opp_avg_elo_change,
        gym_avg_avg_elo_change_diff
    FROM
        cte8
    WHERE
        bout_order_in_event = 1
),
cte9 AS (
    SELECT
        t1.bout_id,
        t1.event_id,
        t4.red_fighter_id,
        t4.blue_fighter_id,
        t1.red_gym_id,
        t2.gym_win_rate AS red_gym_win_rate,
        t1.red_gym_elo,
        t2.gym_avg_elo AS red_gym_avg_elo,
        t2.gym_elo_change AS red_gym_elo_change,
        t2.gym_avg_elo_change AS red_gym_avg_elo_change,
        t2.gym_avg_opp_win_rate AS red_gym_avg_opp_win_rate,
        t2.gym_avg_win_rate_diff AS red_gym_avg_win_rate_diff,
        t2.gym_avg_opp_elo AS red_gym_avg_opp_elo,
        t2.gym_avg_elo_diff AS red_gym_avg_elo_diff,
        t2.gym_avg_opp_avg_elo AS red_gym_avg_opp_avg_elo,
        t2.gym_avg_avg_elo_diff AS red_gym_avg_avg_elo_diff,
        t2.gym_avg_opp_elo_change AS red_gym_avg_opp_elo_change,
        t2.gym_avg_elo_change_diff AS red_gym_avg_elo_change_diff,
        t2.gym_avg_opp_avg_elo_change AS red_gym_avg_opp_avg_elo_change,
        t2.gym_avg_avg_elo_change_diff AS red_gym_avg_avg_elo_change_diff,
        t1.blue_gym_id,
        t3.gym_win_rate AS blue_gym_win_rate,
        t1.blue_gym_elo,
        t3.gym_avg_elo AS blue_gym_avg_elo,
        t3.gym_elo_change AS blue_gym_elo_change,
        t3.gym_avg_elo_change AS blue_gym_avg_elo_change,
        t3.gym_avg_opp_win_rate AS blue_gym_avg_opp_win_rate,
        t3.gym_avg_win_rate_diff AS blue_gym_avg_win_rate_diff,
        t3.gym_avg_opp_elo AS blue_gym_avg_opp_elo,
        t3.gym_avg_elo_diff AS blue_gym_avg_elo_diff,
        t3.gym_avg_opp_avg_elo AS blue_gym_avg_opp_avg_elo,
        t3.gym_avg_avg_elo_diff AS blue_gym_avg_avg_elo_diff,
        t3.gym_avg_opp_elo_change AS blue_gym_avg_opp_elo_change,
        t3.gym_avg_elo_change_diff AS blue_gym_avg_elo_change_diff,
        t3.gym_avg_opp_avg_elo_change AS blue_gym_avg_opp_avg_elo_change,
        t3.gym_avg_avg_elo_change_diff AS blue_gym_avg_avg_elo_change_diff
    FROM
        misc.gym_elo_ratings AS t1
    LEFT JOIN gym_feats AS t2 ON t1.red_gym_id = t2.gym_id AND t1.event_id = t2.event_id
    LEFT JOIN gym_feats AS t3 ON t1.blue_gym_id = t3.gym_id AND t1.event_id = t3.event_id
    INNER JOIN ufcstats_bouts AS t4 ON t1.bout_id = t4.id
),
cte10 AS (
    SELECT
        red_fighter_id AS fighter_id,
        bout_id,
        red_gym_id AS gym_id,
        red_gym_win_rate AS gym_win_rate,
        red_gym_elo AS gym_elo,
        red_gym_avg_elo AS gym_avg_elo,
        red_gym_elo_change AS gym_elo_change,
        red_gym_avg_elo_change AS gym_avg_elo_change,
        red_gym_avg_opp_win_rate AS gym_avg_opp_win_rate,
        red_gym_avg_win_rate_diff AS gym_avg_win_rate_diff,
        red_gym_avg_opp_elo AS gym_avg_opp_elo,
        red_gym_avg_elo_diff AS gym_avg_elo_diff,
        red_gym_avg_opp_avg_elo AS gym_avg_opp_avg_elo,
        red_gym_avg_avg_elo_diff AS gym_avg_avg_elo_diff,
        red_gym_avg_opp_elo_change AS gym_avg_opp_elo_change,
        red_gym_avg_elo_change_diff AS gym_avg_elo_change_diff,
        red_gym_avg_opp_avg_elo_change AS gym_avg_opp_avg_elo_change,
        red_gym_avg_avg_elo_change_diff AS gym_avg_avg_elo_change_diff
    FROM
        cte9
    UNION
    SELECT
        blue_fighter_id AS fighter_id,
        bout_id,
        blue_gym_id AS gym_id,
        blue_gym_win_rate AS gym_win_rate,
        blue_gym_elo AS gym_elo,
        blue_gym_avg_elo AS gym_avg_elo,
        blue_gym_elo_change AS gym_elo_change,
        blue_gym_avg_elo_change AS gym_avg_elo_change,
        blue_gym_avg_opp_win_rate AS gym_avg_opp_win_rate,
        blue_gym_avg_win_rate_diff AS gym_avg_win_rate_diff,
        blue_gym_avg_opp_elo AS gym_avg_opp_elo,
        blue_gym_avg_elo_diff AS gym_avg_elo_diff,
        blue_gym_avg_opp_avg_elo AS gym_avg_opp_avg_elo,
        blue_gym_avg_avg_elo_diff AS gym_avg_avg_elo_diff,
        blue_gym_avg_opp_elo_change AS gym_avg_opp_elo_change,
        blue_gym_avg_elo_change_diff AS gym_avg_elo_change_diff,
        blue_gym_avg_opp_avg_elo_change AS gym_avg_opp_avg_elo_change,
        blue_gym_avg_avg_elo_change_diff AS gym_avg_avg_elo_change_diff
    FROM
        cte9
),
cte11 AS (
    SELECT
        red_fighter_id AS fighter_id,
        id AS bout_id,
        CASE
            WHEN red_outcome = 'W' THEN 1
            ELSE 0
        END AS win_flag
    FROM
        ufcstats_bouts
    UNION
    SELECT
        blue_fighter_id AS fighter_id,
        id AS bout_id,
        CASE
            WHEN blue_outcome = 'W' THEN 1
            ELSE 0
        END AS win_flag
    FROM
        ufcstats_bouts
),
cte12 AS (
    SELECT
        t1.fighter_id,
        t1.bout_id,
        t1.'order',
        t1.opponent_id,
        t2.win_flag,
        t3.gym_id,
        t3.gym_win_rate,
        t3.gym_elo,
        t3.gym_avg_elo,
        t3.gym_elo_change,
        t3.gym_avg_elo_change,
        t3.gym_avg_opp_win_rate,
        t3.gym_avg_win_rate_diff,
        t3.gym_avg_opp_elo,
        t3.gym_avg_elo_diff,
        t3.gym_avg_opp_avg_elo,
        t3.gym_avg_avg_elo_diff,
        t3.gym_avg_opp_elo_change,
        t3.gym_avg_elo_change_diff,
        t3.gym_avg_opp_avg_elo_change,
        t3.gym_avg_avg_elo_change_diff
    FROM
        ufcstats_fighter_histories AS t1
    INNER JOIN cte11 AS t2 ON t1.fighter_id = t2.fighter_id AND t1.bout_id = t2.bout_id
    INNER JOIN cte10 AS t3 ON t1.fighter_id = t3.fighter_id AND t1.bout_id = t3.bout_id
),
cte13 AS (
    SELECT
        t1.*,
        AVG(t1.gym_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_win_rate,
        AVG(t1.gym_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_elo,
        AVG(t1.gym_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_elo,
        AVG(t1.gym_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_elo_change,
        AVG(t1.gym_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_elo_change,
        AVG(t1.gym_avg_opp_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_opp_win_rate,
        AVG(t1.gym_avg_win_rate_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_win_rate_diff,
        AVG(t1.gym_avg_opp_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_opp_elo,
        AVG(t1.gym_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_elo_diff,
        AVG(t1.gym_avg_opp_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_opp_avg_elo,
        AVG(t1.gym_avg_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_avg_elo_diff,
        AVG(t1.gym_avg_opp_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_opp_elo_change,
        AVG(t1.gym_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_elo_change_diff,
        AVG(t1.gym_avg_opp_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_opp_avg_elo_change,
        AVG(t1.gym_avg_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_gym_avg_avg_elo_change_diff,
        AVG(t1.win_flag) OVER (
            PARTITION BY t1.fighter_id, t1.gym_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_win_rate_within_gym,
        AVG(t1.win_flag) OVER (
            PARTITION BY t1.fighter_id, t2.gym_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_win_rate_against_gym
    FROM
        cte12 AS t1
    LEFT JOIN
        cte12 AS t2 ON t1.fighter_id = t2.opponent_id AND t1.bout_id = t2.bout_id AND t1.opponent_id = t2.fighter_id
),
cte14 AS (
    SELECT
        t1.*,
        AVG(t2.gym_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_win_rate,
        AVG(t1.gym_win_rate - t2.gym_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_win_rate_diff,
        AVG(t2.gym_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_elo,
        AVG(t1.gym_avg_elo - t2.gym_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_elo_diff,
        AVG(t2.gym_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_elo_change,
        AVG(t1.gym_elo_change - t2.gym_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_elo_change_diff,
        AVG(t2.gym_avg_opp_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_opp_win_rate,
        AVG(t1.gym_avg_opp_win_rate - t2.gym_avg_opp_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_win_rate_diff,
        AVG(t2.gym_avg_opp_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_opp_elo,
        AVG(t1.gym_avg_opp_elo - t2.gym_avg_opp_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_opp_elo_diff,
        AVG(t2.gym_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_elo_diff,
        AVG(t1.gym_avg_elo_diff - t2.gym_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_elo_diff_diff,
        AVG(t2.gym_avg_opp_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_opp_avg_elo,
        AVG(t1.gym_avg_opp_avg_elo - t2.gym_avg_opp_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_opp_avg_elo_diff,
        AVG(t2.gym_avg_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_avg_elo_diff,
        AVG(t1.gym_avg_avg_elo_diff - t2.gym_avg_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_avg_elo_diff_diff,
        AVG(t2.gym_avg_opp_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_opp_elo_change,
        AVG(t1.gym_avg_opp_elo_change - t2.gym_avg_opp_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_opp_elo_change_diff,
        AVG(t2.gym_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_elo_change_diff,
        AVG(t1.gym_avg_elo_change_diff - t2.gym_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_elo_change_diff_diff,
        AVG(t2.gym_avg_opp_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_opp_avg_elo_change,
        AVG(t1.gym_avg_opp_avg_elo_change - t2.gym_avg_opp_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_opp_avg_elo_change_diff,
        AVG(t2.gym_avg_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_avg_opp_gym_avg_avg_elo_change_diff,
        AVG(t1.gym_avg_avg_elo_change_diff - t2.gym_avg_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_gym_avg_avg_elo_change_diff_diff,
        AVG(t2.fighter_avg_gym_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_win_rate,
        AVG(t1.fighter_avg_gym_win_rate - t2.fighter_avg_gym_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_win_rate_diff,
        AVG(t2.fighter_avg_gym_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_elo,
        AVG(t1.fighter_avg_gym_elo - t2.fighter_avg_gym_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_elo_diff,
        AVG(t2.fighter_avg_gym_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_elo,
        AVG(t1.fighter_avg_gym_avg_elo - t2.fighter_avg_gym_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_elo_diff,
        AVG(t2.fighter_avg_gym_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_elo_change,
        AVG(t1.fighter_avg_gym_elo_change - t2.fighter_avg_gym_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_elo_change_diff,
        AVG(t2.fighter_avg_gym_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_elo_change,
        AVG(t1.fighter_avg_gym_avg_elo_change - t2.fighter_avg_gym_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_elo_change_diff,
        AVG(t2.fighter_avg_gym_avg_opp_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_opp_win_rate,
        AVG(t1.fighter_avg_gym_avg_opp_win_rate - t2.fighter_avg_gym_avg_opp_win_rate) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_opp_win_rate_diff,
        AVG(t2.fighter_avg_gym_avg_win_rate_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_win_rate_diff,
        AVG(t1.fighter_avg_gym_avg_win_rate_diff - t2.fighter_avg_gym_avg_win_rate_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_win_rate_diff_diff,
        AVG(t2.fighter_avg_gym_avg_opp_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_opp_elo,
        AVG(t1.fighter_avg_gym_avg_opp_elo - t2.fighter_avg_gym_avg_opp_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_opp_elo_diff,
        AVG(t2.fighter_avg_gym_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_elo_diff,
        AVG(t1.fighter_avg_gym_avg_elo_diff - t2.fighter_avg_gym_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_elo_diff_diff,
        AVG(t2.fighter_avg_gym_avg_opp_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_opp_avg_elo,
        AVG(t1.fighter_avg_gym_avg_opp_avg_elo - t2.fighter_avg_gym_avg_opp_avg_elo) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_diff,
        AVG(t2.fighter_avg_gym_avg_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_avg_elo_diff,
        AVG(t1.fighter_avg_gym_avg_avg_elo_diff - t2.fighter_avg_gym_avg_avg_elo_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_avg_elo_diff_diff,
        AVG(t2.fighter_avg_gym_avg_opp_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_opp_elo_change,
        AVG(t1.fighter_avg_gym_avg_opp_elo_change - t2.fighter_avg_gym_avg_opp_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_opp_elo_change_diff,
        AVG(t2.fighter_avg_gym_avg_opp_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_opp_avg_elo_change,
        AVG(t1.fighter_avg_gym_avg_opp_avg_elo_change - t2.fighter_avg_gym_avg_opp_avg_elo_change) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_change_diff,
        AVG(t2.fighter_avg_gym_avg_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_avg_gym_avg_avg_elo_change_diff,
        AVG(t1.fighter_avg_gym_avg_avg_elo_change_diff - t2.fighter_avg_gym_avg_avg_elo_change_diff) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_avg_gym_avg_avg_elo_change_diff_diff,
        AVG(t2.fighter_win_rate_within_gym) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_win_rate_within_gym,
        AVG(t1.fighter_win_rate_within_gym - t2.fighter_win_rate_within_gym) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_win_rate_within_gym_diff,
        AVG(t2.fighter_win_rate_against_gym) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS avg_opp_fighter_win_rate_against_gym,
        AVG(t1.fighter_win_rate_against_gym - t2.fighter_win_rate_against_gym) OVER (
            PARTITION BY t1.fighter_id
            ORDER BY t1.'order'
            ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
        ) AS fighter_opp_avg_fighter_win_rate_against_gym_diff
    FROM
        cte13 AS t1
    LEFT JOIN
        cte13 AS t2 ON t1.fighter_id = t2.opponent_id AND t1.bout_id = t2.bout_id AND t1.opponent_id = t2.fighter_id
)
SELECT
    id,
    t2.gym_win_rate - t3.gym_win_rate AS gym_win_rate_diff,
    t2.gym_elo - t3.gym_elo AS gym_elo_diff,
    t2.gym_avg_elo - t3.gym_avg_elo AS gym_avg_elo_diff,
    t2.gym_elo_change - t3.gym_elo_change AS gym_elo_change_diff,
    t2.gym_avg_elo_change - t3.gym_avg_elo_change AS gym_avg_elo_change_diff,
    t2.gym_avg_opp_win_rate - t3.gym_avg_opp_win_rate AS gym_avg_opp_win_rate_diff,
    t2.gym_avg_win_rate_diff - t3.gym_avg_win_rate_diff AS gym_avg_win_rate_diff_diff,
    t2.gym_avg_opp_elo - t3.gym_avg_opp_elo AS gym_avg_opp_elo_diff,
    t2.gym_avg_elo_diff - t3.gym_avg_elo_diff AS gym_avg_elo_diff_diff,
    t2.gym_avg_opp_avg_elo - t3.gym_avg_opp_avg_elo AS gym_avg_opp_avg_elo_diff,
    t2.gym_avg_avg_elo_diff - t3.gym_avg_avg_elo_diff AS gym_avg_avg_elo_diff_diff,
    t2.gym_avg_opp_elo_change - t3.gym_avg_opp_elo_change AS gym_avg_opp_elo_change_diff,
    t2.gym_avg_elo_change_diff - t3.gym_avg_elo_change_diff AS gym_avg_elo_change_diff_diff,
    t2.gym_avg_opp_avg_elo_change - t3.gym_avg_opp_avg_elo_change AS gym_avg_opp_avg_elo_change_diff,
    t2.gym_avg_avg_elo_change_diff - t3.gym_avg_avg_elo_change_diff AS gym_avg_avg_elo_change_diff_diff,
    t2.fighter_avg_gym_win_rate - t3.fighter_avg_gym_win_rate AS fighter_avg_gym_win_rate_diff,
    t2.fighter_avg_gym_elo - t3.fighter_avg_gym_elo AS fighter_avg_gym_elo_diff,
    t2.fighter_avg_gym_avg_elo - t3.fighter_avg_gym_avg_elo AS fighter_avg_gym_avg_elo_diff,
    t2.fighter_avg_gym_elo_change - t3.fighter_avg_gym_elo_change AS fighter_avg_gym_elo_change_diff,
    t2.fighter_avg_gym_avg_elo_change - t3.fighter_avg_gym_avg_elo_change AS fighter_avg_gym_avg_elo_change_diff,
    t2.fighter_avg_gym_avg_opp_win_rate - t3.fighter_avg_gym_avg_opp_win_rate AS fighter_avg_gym_avg_opp_win_rate_diff,
    t2.fighter_avg_gym_avg_win_rate_diff - t3.fighter_avg_gym_avg_win_rate_diff AS fighter_avg_gym_avg_win_rate_diff_diff,
    t2.fighter_avg_gym_avg_opp_elo - t3.fighter_avg_gym_avg_opp_elo AS fighter_avg_gym_avg_opp_elo_diff,
    t2.fighter_avg_gym_avg_elo_diff - t3.fighter_avg_gym_avg_elo_diff AS fighter_avg_gym_avg_elo_diff_diff,
    t2.fighter_avg_gym_avg_opp_avg_elo - t3.fighter_avg_gym_avg_opp_avg_elo AS fighter_avg_gym_avg_opp_avg_elo_diff,
    t2.fighter_avg_gym_avg_avg_elo_diff - t3.fighter_avg_gym_avg_avg_elo_diff AS fighter_avg_gym_avg_avg_elo_diff_diff,
    t2.fighter_avg_gym_avg_opp_elo_change - t3.fighter_avg_gym_avg_opp_elo_change AS fighter_avg_gym_avg_opp_elo_change_diff,
    t2.fighter_avg_gym_avg_elo_change_diff - t3.fighter_avg_gym_avg_elo_change_diff AS fighter_avg_gym_avg_elo_change_diff_diff,
    t2.fighter_avg_gym_avg_opp_avg_elo_change - t3.fighter_avg_gym_avg_opp_avg_elo_change AS fighter_avg_gym_avg_opp_avg_elo_change_diff,
    t2.fighter_avg_gym_avg_avg_elo_change_diff - t3.fighter_avg_gym_avg_avg_elo_change_diff AS fighter_avg_gym_avg_avg_elo_change_diff_diff,
    t2.fighter_win_rate_within_gym - t3.fighter_win_rate_within_gym AS fighter_win_rate_within_gym_diff,
    t2.fighter_win_rate_against_gym - t3.fighter_win_rate_against_gym AS fighter_win_rate_against_gym_diff,
    t2.fighter_avg_opp_gym_win_rate - t3.fighter_avg_opp_gym_win_rate AS fighter_avg_opp_gym_win_rate_diff,
    t2.fighter_opp_avg_gym_win_rate_diff - t3.fighter_opp_avg_gym_win_rate_diff AS fighter_opp_avg_gym_win_rate_diff_diff,
    t2.fighter_avg_opp_gym_avg_elo - t3.fighter_avg_opp_gym_avg_elo AS fighter_avg_opp_gym_avg_elo_diff,
    t2.fighter_opp_avg_gym_avg_elo_diff - t3.fighter_opp_avg_gym_avg_elo_diff AS fighter_opp_avg_gym_avg_elo_diff_diff,
    t2.fighter_avg_opp_gym_elo_change - t3.fighter_avg_opp_gym_elo_change AS fighter_avg_opp_gym_elo_change_diff,
    t2.fighter_opp_avg_gym_elo_change_diff - t3.fighter_opp_avg_gym_elo_change_diff AS fighter_opp_avg_gym_elo_change_diff_diff,
    t2.fighter_avg_opp_gym_avg_opp_win_rate - t3.fighter_avg_opp_gym_avg_opp_win_rate AS fighter_avg_opp_gym_avg_opp_win_rate_diff,
    t2.fighter_opp_avg_gym_avg_win_rate_diff - t3.fighter_opp_avg_gym_avg_win_rate_diff AS fighter_opp_avg_gym_avg_win_rate_diff_diff,
    t2.fighter_avg_opp_gym_avg_opp_elo - t3.fighter_avg_opp_gym_avg_opp_elo AS fighter_avg_opp_gym_avg_opp_elo_diff,
    t2.fighter_opp_avg_gym_avg_opp_elo_diff - t3.fighter_opp_avg_gym_avg_opp_elo_diff AS fighter_opp_avg_gym_avg_opp_elo_diff_diff,
    t2.fighter_avg_opp_gym_avg_elo_diff - t3.fighter_avg_opp_gym_avg_elo_diff AS fighter_avg_opp_gym_avg_elo_diff_diff,
    t2.fighter_opp_avg_gym_avg_elo_diff_diff - t3.fighter_opp_avg_gym_avg_elo_diff_diff AS fighter_opp_avg_gym_avg_elo_diff_diff_diff,
    t2.fighter_avg_opp_gym_avg_opp_avg_elo - t3.fighter_avg_opp_gym_avg_opp_avg_elo AS fighter_avg_opp_gym_avg_opp_avg_elo_diff,
    t2.fighter_opp_avg_gym_avg_opp_avg_elo_diff - t3.fighter_opp_avg_gym_avg_opp_avg_elo_diff AS fighter_opp_avg_gym_avg_opp_avg_elo_diff_diff,
    t2.fighter_avg_opp_gym_avg_avg_elo_diff - t3.fighter_avg_opp_gym_avg_avg_elo_diff AS fighter_avg_opp_gym_avg_avg_elo_diff_diff,
    t2.fighter_opp_avg_gym_avg_avg_elo_diff_diff - t3.fighter_opp_avg_gym_avg_avg_elo_diff_diff AS fighter_opp_avg_gym_avg_avg_elo_diff_diff_diff,
    t2.fighter_avg_opp_gym_avg_opp_elo_change - t3.fighter_avg_opp_gym_avg_opp_elo_change AS fighter_avg_opp_gym_avg_opp_elo_change_diff,
    t2.fighter_opp_avg_gym_avg_opp_elo_change_diff - t3.fighter_opp_avg_gym_avg_opp_elo_change_diff AS fighter_opp_avg_gym_avg_opp_elo_change_diff_diff,
    t2.fighter_avg_opp_gym_avg_elo_change_diff - t3.fighter_avg_opp_gym_avg_elo_change_diff AS fighter_avg_opp_gym_avg_elo_change_diff_diff,
    t2.fighter_opp_avg_gym_avg_elo_change_diff_diff - t3.fighter_opp_avg_gym_avg_elo_change_diff_diff AS fighter_opp_avg_gym_avg_elo_change_diff_diff_diff,
    t2.fighter_avg_opp_gym_avg_opp_avg_elo_change - t3.fighter_avg_opp_gym_avg_opp_avg_elo_change AS fighter_avg_opp_gym_avg_opp_avg_elo_change_diff,
    t2.fighter_opp_avg_gym_avg_opp_avg_elo_change_diff - t3.fighter_opp_avg_gym_avg_opp_avg_elo_change_diff AS fighter_opp_avg_gym_avg_opp_avg_elo_change_diff_diff,
    t2.fighter_avg_opp_gym_avg_avg_elo_change_diff - t3.fighter_avg_opp_gym_avg_avg_elo_change_diff AS fighter_avg_opp_gym_avg_avg_elo_change_diff_diff,
    t2.fighter_opp_avg_gym_avg_avg_elo_change_diff_diff - t3.fighter_opp_avg_gym_avg_avg_elo_change_diff_diff AS fighter_opp_avg_gym_avg_avg_elo_change_diff_diff_diff,
    t2.avg_opp_fighter_avg_gym_win_rate - t3.avg_opp_fighter_avg_gym_win_rate AS avg_opp_fighter_avg_gym_win_rate_diff,
    t2.fighter_opp_avg_fighter_avg_gym_win_rate_diff - t3.fighter_opp_avg_fighter_avg_gym_win_rate_diff AS fighter_opp_avg_fighter_avg_gym_win_rate_diff_diff,
    t2.avg_opp_fighter_avg_gym_elo - t3.avg_opp_fighter_avg_gym_elo AS avg_opp_fighter_avg_gym_elo_diff,
    t2.fighter_opp_avg_fighter_avg_gym_elo_diff - t3.fighter_opp_avg_fighter_avg_gym_elo_diff AS fighter_opp_avg_fighter_avg_gym_elo_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_elo - t3.avg_opp_fighter_avg_gym_avg_elo AS avg_opp_fighter_avg_gym_avg_elo_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_elo_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_elo_diff AS fighter_opp_avg_fighter_avg_gym_avg_elo_diff_diff,
    t2.avg_opp_fighter_avg_gym_elo_change - t3.avg_opp_fighter_avg_gym_elo_change AS avg_opp_fighter_avg_gym_elo_change_diff,
    t2.fighter_opp_avg_fighter_avg_gym_elo_change_diff - t3.fighter_opp_avg_fighter_avg_gym_elo_change_diff AS fighter_opp_avg_fighter_avg_gym_elo_change_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_elo_change - t3.avg_opp_fighter_avg_gym_avg_elo_change AS avg_opp_fighter_avg_gym_avg_elo_change_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_elo_change_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_elo_change_diff AS fighter_opp_avg_fighter_avg_gym_avg_elo_change_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_opp_win_rate - t3.avg_opp_fighter_avg_gym_avg_opp_win_rate AS avg_opp_fighter_avg_gym_avg_opp_win_rate_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_opp_win_rate_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_opp_win_rate_diff AS fighter_opp_avg_fighter_avg_gym_avg_opp_win_rate_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_win_rate_diff - t3.avg_opp_fighter_avg_gym_avg_win_rate_diff AS avg_opp_fighter_avg_gym_avg_win_rate_diff_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_win_rate_diff_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_win_rate_diff_diff AS fighter_opp_avg_fighter_avg_gym_avg_win_rate_diff_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_opp_elo - t3.avg_opp_fighter_avg_gym_avg_opp_elo AS avg_opp_fighter_avg_gym_avg_opp_elo_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_opp_elo_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_opp_elo_diff AS fighter_opp_avg_fighter_avg_gym_avg_opp_elo_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_elo_diff - t3.avg_opp_fighter_avg_gym_avg_elo_diff AS avg_opp_fighter_avg_gym_avg_elo_diff_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_elo_diff_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_elo_diff_diff AS fighter_opp_avg_fighter_avg_gym_avg_elo_diff_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_opp_avg_elo - t3.avg_opp_fighter_avg_gym_avg_opp_avg_elo AS avg_opp_fighter_avg_gym_avg_opp_avg_elo_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_diff AS fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_avg_elo_diff - t3.avg_opp_fighter_avg_gym_avg_avg_elo_diff AS avg_opp_fighter_avg_gym_avg_avg_elo_diff_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_avg_elo_diff_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_avg_elo_diff_diff AS fighter_opp_avg_fighter_avg_gym_avg_avg_elo_diff_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_opp_elo_change - t3.avg_opp_fighter_avg_gym_avg_opp_elo_change AS avg_opp_fighter_avg_gym_avg_opp_elo_change_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_opp_elo_change_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_opp_elo_change_diff AS fighter_opp_avg_fighter_avg_gym_avg_opp_elo_change_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_opp_avg_elo_change - t3.avg_opp_fighter_avg_gym_avg_opp_avg_elo_change AS avg_opp_fighter_avg_gym_avg_opp_avg_elo_change_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_change_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_change_diff AS fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_change_diff_diff,
    t2.avg_opp_fighter_avg_gym_avg_avg_elo_change_diff - t3.avg_opp_fighter_avg_gym_avg_avg_elo_change_diff AS avg_opp_fighter_avg_gym_avg_avg_elo_change_diff_diff,
    t2.fighter_opp_avg_fighter_avg_gym_avg_avg_elo_change_diff_diff - t3.fighter_opp_avg_fighter_avg_gym_avg_avg_elo_change_diff_diff AS fighter_opp_avg_fighter_avg_gym_avg_avg_elo_change_diff_diff_diff,
    t2.avg_opp_fighter_win_rate_within_gym - t3.avg_opp_fighter_win_rate_within_gym AS avg_opp_fighter_win_rate_within_gym_diff,
    t2.fighter_opp_avg_fighter_win_rate_within_gym_diff - t3.fighter_opp_avg_fighter_win_rate_within_gym_diff AS fighter_opp_avg_fighter_win_rate_within_gym_diff_diff,
    t2.avg_opp_fighter_win_rate_against_gym - t3.avg_opp_fighter_win_rate_against_gym AS avg_opp_fighter_win_rate_against_gym_diff,
    t2.fighter_opp_avg_fighter_win_rate_against_gym_diff - t3.fighter_opp_avg_fighter_win_rate_against_gym_diff AS fighter_opp_avg_fighter_win_rate_against_gym_diff_diff,
    CASE
        WHEN red_outcome = 'W' THEN 1
        ELSE 0
    END AS red_win
FROM ufcstats_bouts AS t1
LEFT JOIN cte14 AS t2 ON t1.red_fighter_id = t2.fighter_id AND t1.id = t2.bout_id
LEFT JOIN cte14 AS t3 ON t1.blue_fighter_id = t3.fighter_id AND t1.id = t3.bout_id
WHERE event_id IN (
        SELECT id
        FROM ufcstats_events
        WHERE is_ufc_event = 1
            AND date >= '2008-04-19'
    );
"""

data_dir = os.path.join(os.path.dirname("__file__"), "..", "..", "data")
misc_db_path = os.path.join(data_dir, "misc.db")

with sqlite3.connect(db_path) as conn:
    conn.execute(f"ATTACH DATABASE '{misc_db_path}' AS misc")
    df = pd.read_sql(query, conn)
df

,id,gym_win_rate_diff,gym_elo_diff,gym_avg_elo_diff,gym_elo_change_diff,gym_avg_elo_change_diff,gym_avg_opp_win_rate_diff,gym_avg_win_rate_diff_diff,gym_avg_opp_elo_diff,gym_avg_elo_diff_diff,...,fighter_opp_avg_fighter_avg_gym_avg_opp_elo_change_diff_diff,avg_opp_fighter_avg_gym_avg_opp_avg_elo_change_diff,fighter_opp_avg_fighter_avg_gym_avg_opp_avg_elo_change_diff_diff,avg_opp_fighter_avg_gym_avg_avg_elo_change_diff_diff,fighter_opp_avg_fighter_avg_gym_avg_avg_elo_change_diff_diff_diff,avg_opp_fighter_win_rate_within_gym_diff,fighter_opp_avg_fighter_win_rate_within_gym_diff_diff,avg_opp_fighter_win_rate_against_gym_diff,fighter_opp_avg_fighter_win_rate_against_gym_diff_diff,red_win
0,be38ed9ccfe2ee03,0.166667,21.561187,17.125557,83.926737,-22.202119,0.037333,0.200048,-87.425891,104.551448,...,-89.354546,39.599304,-48.404790,19.417505,NaN,-0.128889,0.258333,NaN,NaN,1
1,eb1b371dfc37fcdb,NaN,378.723213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,219bd976b8ca745d,0.238095,102.823023,103.564798,93.656572,6.608357,0.177038,0.375060,44.878900,46.142172,...,-34.803622,33.147702,24.738000,-38.715103,NaN,0.291111,0.445833,NaN,NaN,0
3,af178adff964d854,-0.424242,182.661888,95.445843,-14.938077,NaN,-0.100330,NaN,-62.681004,160.865729,...,NaN,1.336358,NaN,-49.614958,NaN,0.194444,NaN,NaN,NaN,0
4,920194911d727a38,NaN,249.355147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,5238f6470d0557fb,-0.227778,-184.827291,-69.001319,7.213307,-25.502875,0.019437,-0.241346,-30.537902,-38.463417,...,13.172328,0.288681,9.918885,11.062809,-39.431016,0.086851,0.109256,NaN,NaN,0
7072,7b1bc4ff776f12c1,0.137263,-14.244733,92.573952,113.115428,1.571874,-0.010170,0.144036,9.259230,87.409904,...,20.667869,-0.166847,12.721613,-9.566957,2.104026,-0.180293,0.603531,NaN,NaN,0
7073,1a635a5e4551e7d5,-0.087787,-130.412623,-82.483562,-50.443838,-0.544245,0.045328,-0.139801,-10.812760,-78.827513,...,1.015473,1.187024,6.207555,27.783345,-58.652528,0.411111,-0.531481,NaN,NaN,1
7074,7521015554088962,NaN,65.688581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.334003,NaN,0.792922,NaN,0.084607,-0.383766,-0.166667,NaN,1


In [45]:
class GymEloSystem:
    def __init__(self, k_factor: int = 100):
        self.data_dir = os.path.join(os.path.dirname("__file__"), "..", "..", "data")
        self.db_path = os.path.join(self.data_dir, "ufc.db")
        self.misc_db_path = os.path.join(self.data_dir, "misc.db")

        self.k_factor = k_factor
        self.elo_ratings = {}

    def create_misc_db(self) -> None:
        with sqlite3.connect(self.misc_db_path) as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS gym_elo_ratings (
                    bout_id TEXT PRIMARY KEY,
                    event_id TEXT NOT NULL,
                    red_outcome TEXT NOT NULL,
                    red_gym_id TEXT,
                    blue_gym_id TEXT,
                    red_gym_elo REAL,
                    blue_gym_elo REAL           
                );
            """)
            conn.commit()
    
    def get_gym_matchups(self) -> pd.DataFrame:
        query = """
        WITH cte1 AS (
            SELECT
                fighter_id,
                bout_id,
                CASE
                    WHEN gym_id IS NOT NULL THEN gym_id
                    ELSE gym_name
                END AS gym_id,
                ROW_NUMBER() OVER (PARTITION BY fighter_id, bout_id ORDER BY t1.rowid) AS gym_rank
            FROM
                tapology_fighter_gyms t1
            WHERE
                gym_purpose = 'Primary'
        ),
        cte2 AS (
            SELECT
                *,
                ROW_NUMBER() OVER (PARTITION BY fighter_id, bout_id ORDER BY gym_rank) AS primary_gym_rank
            FROM
                cte1 AS t1
        ),
        cte3 AS (
            SELECT
                *
            FROM cte2
            WHERE primary_gym_rank = 1
        ),
        cte4 AS (
            SELECT
                t1.fighter_id,
                t1.bout_id,
                CASE
                    WHEN t1.gym_id IS NOT NULL THEN t1.gym_id
                    ELSE t1.gym_name
                END AS gym_id,
                COUNT(t1.gym_name) OVER (PARTITION BY t1.fighter_id, t1.bout_id) AS gym_count,
                ROW_NUMBER() OVER (PARTITION BY t1.fighter_id, t1.bout_id ORDER BY t1.rowid) AS gym_rank
            FROM
                tapology_fighter_gyms AS t1
        ),
        cte5 AS (
            SELECT
                t1.fighter_id,
                t1.bout_id,
                t1.gym_id,
                t1.gym_count,
                t1.gym_rank,
                CASE
                    WHEN t2.fighter_id IS NOT NULL AND t2.bout_id IS NOT NULL THEN 1
                    ELSE 0
                END AS has_primary_flag,
                t3.primary_gym_rank
            FROM
                cte4 AS t1
            LEFT JOIN cte3 AS t2 ON t1.fighter_id = t2.fighter_id AND t1.bout_id = t2.bout_id
            LEFT JOIN cte3 AS t3 ON t1.fighter_id = t3.fighter_id AND t1.bout_id = t3.bout_id AND t1.gym_id = t3.gym_id
        ),
        cte6 AS (
            SELECT
                fighter_id,
                bout_id,
                gym_id
            FROM
                cte5
            WHERE
                gym_count = 1
            UNION
            SELECT
                fighter_id,
                bout_id,
                gym_id
            FROM
                cte5
            WHERE
                gym_count > 1 AND has_primary_flag = 1 AND primary_gym_rank = 1
            UNION
            SELECT
                fighter_id,
                bout_id,
                gym_id
            FROM
                cte5
            WHERE
                gym_count > 1 AND has_primary_flag = 0 AND gym_rank = 1
        ),
        fighter_gyms AS (
            SELECT
                t2.ufcstats_id AS fighter_id,
                t3.ufcstats_id AS bout_id,
                CASE
                    WHEN t4.parent_id IS NOT NULL THEN t4.parent_id
                    ELSE t1.gym_id
                END AS gym_id
            FROM
                cte6 AS t1
            INNER JOIN fighter_mapping AS t2 ON t1.fighter_id = t2.tapology_id
            INNER JOIN bout_mapping AS t3 ON t1.bout_id = t3.tapology_id
            LEFT JOIN tapology_gyms AS t4 ON t1.gym_id = t4.id
        )
        SELECT
            t1.id AS bout_id,
            t1.event_id,
            t1.red_outcome,
            t3.gym_id AS red_gym_id,
            t4.gym_id AS blue_gym_id
        FROM
            ufcstats_bouts AS t1
        INNER JOIN
            event_mapping AS t2 ON t1.event_id = t2.ufcstats_id
        LEFT JOIN
            fighter_gyms AS t3 ON t3.fighter_id = t1.red_fighter_id AND t3.bout_id = t1.id
        LEFT JOIN
            fighter_gyms AS t4 ON t4.fighter_id = t1.blue_fighter_id AND t4.bout_id = t1.id
        ORDER BY
            t2.wikipedia_id, t1.bout_order
        """

        with sqlite3.connect(self.db_path) as conn:
            df = pd.read_sql_query(query, conn)
        
        return df

    def initialize_gym(self, gym_id: str) -> None:
        if gym_id not in self.elo_ratings and gym_id is not None:
            self.elo_ratings[gym_id] = 1000.0
    
    def get_median_rating(self):
        if len(self.elo_ratings) == 0:
            return 1000.0
        
        elo_values = np.array(list(self.elo_ratings.values()))
        median_rating = np.median(elo_values)

        return median_rating
    
    def calculate_expected_score(self, rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    
    def update_gym_rating(self, gym_id: str, expected_score_total, actual_score_total) -> None:
        if gym_id is not None:
            self.elo_ratings[gym_id] += self.k_factor * (actual_score_total - expected_score_total)

    def simulate_event(self, event_df: pd.DataFrame) -> pd.DataFrame:
        gym_ids = set(event_df["red_gym_id"]) | set(event_df["blue_gym_id"])
        for gym_id in gym_ids:
            self.initialize_gym(gym_id)
        
        red_gym_elo_list = []
        blue_gym_elo_list = []

        expected_scores = {}
        actual_scores = {}
        for _, row in event_df.iterrows():
            red_gym_id = row["red_gym_id"]
            blue_gym_id = row["blue_gym_id"]
            red_outcome = row["red_outcome"]

            red_gym_elo = self.elo_ratings[red_gym_id] if red_gym_id is not None else self.get_median_rating()
            blue_gym_elo = self.elo_ratings[blue_gym_id] if blue_gym_id is not None else self.get_median_rating()
            red_gym_elo_list.append(red_gym_elo)
            blue_gym_elo_list.append(blue_gym_elo)

            if red_outcome == "NC":
                continue

            red_expected_score = self.calculate_expected_score(red_gym_elo, blue_gym_elo)
            blue_expected_score = self.calculate_expected_score(blue_gym_elo, red_gym_elo)
            red_actual_score = 1 if red_outcome == "W" else 0.5 if red_outcome == "D" else 0
            blue_actual_score = 1 - red_actual_score

            if red_gym_id is not None:
                expected_scores[red_gym_id] = expected_scores.get(red_gym_id, 0) + red_expected_score
                actual_scores[red_gym_id] = actual_scores.get(red_gym_id, 0) + red_actual_score
            
            if blue_gym_id is not None:
                expected_scores[blue_gym_id] = expected_scores.get(blue_gym_id, 0) + blue_expected_score
                actual_scores[blue_gym_id] = actual_scores.get(blue_gym_id, 0) + blue_actual_score

        result_df = event_df.copy()
        result_df["red_gym_elo"] = red_gym_elo_list
        result_df["blue_gym_elo"] = blue_gym_elo_list

        assert len(expected_scores) == len(actual_scores), "Expected and actual scores mismatch"
        for gym_id in expected_scores.keys():
            expected_score_total = expected_scores[gym_id]
            actual_score_total = actual_scores[gym_id]

            self.update_gym_rating(gym_id, expected_score_total, actual_score_total)
        
        return result_df

    def simulate_all_events(self, df: pd.DataFrame):
        event_ids = df["event_id"].unique()
        result_list = []
        for event_id in event_ids:
            event_df = df.loc[df["event_id"] == event_id].copy()
            result_df = self.simulate_event(event_df)
            result_list.append(result_df)
        
        result_df = pd.concat(result_list, ignore_index=True).reset_index(drop=True)

        # with sqlite3.connect(self.misc_db_path) as conn:
        #     result_df.to_sql("gym_elo_ratings", conn, if_exists="replace", index=False)
        #     conn.commit()

        return result_df

In [48]:
gym_elo_system = GymEloSystem()
gym_matchups_df = gym_elo_system.get_gym_matchups()
result_df = gym_elo_system.simulate_all_events(gym_matchups_df)
result_df

,bout_id,event_id,red_outcome,red_gym_id,blue_gym_id,red_gym_elo,blue_gym_elo
0,567a09fd200cfa05,6420efac0578988b,W,Kamakura,None,1000.000000,1000.000000
1,2d2bbc86e941e05c,6420efac0578988b,W,None,None,1000.000000,1000.000000
2,cecdc0da584274b9,6420efac0578988b,W,831-gracie-humaita,None,1000.000000,1000.000000
3,46acd54cc0c905fb,6420efac0578988b,W,298-lions-den,None,1000.000000,1000.000000
4,ac7ca2ec38b96c1a,6420efac0578988b,W,Kamakura,None,1000.000000,1000.000000
...,...,...,...,...,...,...,...
7977,5238f6470d0557fb,72c9c2eadfc3277e,L,304-metro-fight-club,1988-los-perros-sarnosos,1073.611981,1258.439272
7978,7b1bc4ff776f12c1,72c9c2eadfc3277e,L,8000-cm-system,612-factoryx-muay-thai,1160.933366,1175.178099
7979,1a635a5e4551e7d5,72c9c2eadfc3277e,W,724-xtreme-couture,451-fight-ready-mma,977.590549,1108.003172
7980,7521015554088962,72c9c2eadfc3277e,W,12259-bloodline-combat-sports,None,1038.729714,973.041133
